# Init

In [ ]:
import matplotlib.pylab as plt

import numpy as np
import pandas as pd
import seaborn as sns

from luescher_nd.zeta.zeta2d import Zeta2D

In [ ]:
%load_ext blackcellmagic

# Comparison with Tom

Init the zeta functions

In [ ]:
N = 40
z_spherical = Zeta2D(N, spherical=True)
z_cartesian = Zeta2D(N, spherical=False)

Plot to compare with Tom

In [ ]:
fig, ax = plt.subplots(figsize=(3,2), dpi=200)

tol = 40.0

x = np.linspace(-4, 10, 1002)

for z, key in zip([z_spherical, z_cartesian], ["spherical", "cartesian"]):
    y = z(x)
    y[y>tol] = np.inf
    y[y<-tol] = -np.inf

    ax.plot(x, y, ls="--", label=key)

    ax.axhline(0, color="black", lw=.5, ls="-", zorder=-1)
    ax.axvline(0, color="black", lw=.5, ls="-", zorder=-1)

sns.despine()

ax.legend(loc="upper left", bbox_to_anchor=(1,1), frameon=False)

ax.set_ylim(-20, 20)
ax.set_xlim(-4, 10)

ax.set_xlabel("$x$")
ax.set_ylabel("$S_2(x)$")

ax.set_title(f"$N={N}$", fontsize=10)

plt.show()

In [ ]:
fig.savefig("figs/zeta-2d.jpg", bbox_inches="tight")

## ERE

Computation of Effective range expansion

In [ ]:
from scipy.optimize import minimize_scalar

Assume fixed scattering length and box volume to solve Tom's equation (26)

In [ ]:
a0 = 1.0
L = 1.0

lhs = 2 * np.pi * np.log(2 * np.pi * a0 / L) 

In [ ]:
class MinKernel:
    def __init__(self, val, func):
        self.val = val
        self.func = func
        
    def chi2(self, x):
        return (self.val - self.func(x))**2

Exactly extrac the ground state by finding the intersections between zeta and $\cot(\delta_o(x))$ (is it the ground state though?)

In [ ]:
kernel_spherical = MinKernel(lhs, z_spherical)
kernel_cartesian = MinKernel(lhs, z_cartesian)

x0_spherical = minimize_scalar(kernel_spherical.chi2, bracket=(0, 0.5), tol=1.e-12).x[0]
x0_cartesian = minimize_scalar(kernel_cartesian.chi2, bracket=(0, 0.5), tol=1.e-12).x[0]

In [ ]:
def ere(y, x):
    return y / np.pi**2 + 1 / np.pi * np.log(x)

Plot intersection

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2), dpi=200)

tol = 80.0
x = np.linspace(0, 10, 1002)

for z, key in zip([z_spherical, z_cartesian], ["spherical", "cartesian"]):
    y = z(x)
    y[y > tol] = np.inf
    y[y < -tol] = -np.inf

    ax.plot(x, ere(y, x), ls="--", label=key)

    ax.axhline(0, color="black", lw=0.5, ls="-", zorder=-1)
    ax.axvline(0, color="black", lw=0.5, ls="-", zorder=-1)

ax.plot(
    x0_spherical,
    ere(z_spherical(x0_spherical), x0_spherical),
    marker="o",
    ms=2,
    color="black",
    zorder=2,
)
ax.plot(
    x0_cartesian,
    ere(z_cartesian(x0_cartesian), x0_cartesian),
    marker="o",
    ms=2,
    color="black",
    zorder=2,
)

sns.despine()

ax.plot(
    x,
    2 / np.pi * np.log(a0 * 2 * np.pi * np.sqrt(x) / L),
    color="green",
    ls="-",
    lw="1",
    label="Analytic",
)

ax.legend(loc="upper left", bbox_to_anchor=(1, 1), frameon=False)

ax.set_ylim(-5, 5)
ax.set_xlim(0, 10)

ax.set_xlabel("$x$")
ax.set_ylabel("$\cot(\delta_0(x))$")

plt.show()


# Fit FV discrete Hamiltonian to ground state

In [ ]:
from luescher_nd.hamiltonians.contact import MomentumContactHamiltonian
from luescher_nd.operators import get_parity_projector

from scipy.sparse.linalg import eigsh

In [ ]:
n1d = 20
epsilon = L / n1d
ndim = 2
nstep = None
mass = 4.758
mu = mass / 2

p_minus = get_parity_projector(n1d, ndim, positive=False)

def get_first_state(contact_strength):

    eigs = np.sort(eigsh(MomentumContactHamiltonian(
        contact_strength=-np.exp(contact_strength),
        n1d=n1d,
        epsilon=epsilon,
        mass=mass,
        ndim=ndim,
        nstep=nstep,
        filter_out=p_minus,
        filter_cutoff=1.0e2,
    ).op, return_eigenvectors=False, k=5, which="SA"))
    return eigs[1]

Note that I do not fit the ground state but the first excited state ($x>0$).

In [ ]:
contact_fitter_spherical = MinKernel(
    x0_spherical * (2 * np.pi / L) ** 2 / 2 / mu, get_first_state
)
contact_fitter_cartesian = MinKernel(
    x0_cartesian * (2 * np.pi / L) ** 2 / 2 / mu, get_first_state
)

c0_spherical = minimize_scalar(contact_fitter_spherical.chi2, tol=1.e-12).x
c0_cartesian = minimize_scalar(contact_fitter_cartesian.chi2, tol=1.e-12).x

# Plot computed ERE

get spectrum

In [ ]:
h_spherical = MomentumContactHamiltonian(
        contact_strength=-np.exp(c0_spherical),
        n1d=n1d,
        epsilon=epsilon,
        mass=mass,
        ndim=ndim,
        nstep=nstep,
        filter_out=p_minus,
        filter_cutoff=1.0e2,
    )
h_cartesian = MomentumContactHamiltonian(
        contact_strength=-np.exp(c0_cartesian),
        n1d=n1d,
        epsilon=epsilon,
        mass=mass,
        ndim=ndim,
        nstep=nstep,
        filter_out=p_minus,
        filter_cutoff=1.0e2,
    )

eigs_spherical = eigsh(h_spherical.op, return_eigenvectors=False, k=20, which="SA")
eigs_caresian = eigsh(h_cartesian.op, return_eigenvectors=False, k=20, which="SA")

In [ ]:
x_spherical = np.sort(eigs_spherical) * 2 * mu * L**2 / 4 / np.pi**2
x_cartesian = np.sort(eigs_caresian) * 2 * mu * L**2 / 4 / np.pi**2

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2), dpi=200)

tol = 80.0
x = np.linspace(0, 10, 1002)

for z, key in zip([z_spherical, z_cartesian], ["spherical", "cartesian"]):
    y = z(x)
    y[y > tol] = np.inf
    y[y < -tol] = -np.inf

    ax.plot(x, ere(y, x), ls="--")

    ax.axhline(0, color="black", lw=0.5, ls="-", zorder=-1)
    ax.axvline(0, color="black", lw=0.5, ls="-", zorder=-1)

ax.plot(
    x_spherical,
    ere(z_spherical(x_spherical), x_spherical),
    marker="o",
    ms=2,
    ls="None",
    color="black",
    zorder=2,
    label="Spectrum Spherical"
)
ax.plot(
    x_cartesian,
    ere(z_cartesian(x_cartesian), x_cartesian),
    marker="s",
    ms=2,
    ls="None",
    color="black",
    zorder=2,
    label="Spectrum Cartesian"
)

sns.despine()

ax.plot(
    x,
    2 / np.pi * np.log(a0 * 2 * np.pi * np.sqrt(x) / L),
    color="green",
    ls="-",
    lw="1",
    label="Analytic",
    zorder=1
)

ax.legend(loc="upper left", bbox_to_anchor=(1, 1), frameon=False)

ax.set_ylim(-5, 5)
ax.set_xlim(0, 10)

ax.set_xlabel("$x$")
ax.set_ylabel("$\cot(\delta_0(x))$")

ax.set_title(f"$\epsilon = {epsilon}$ [fm], $L = {L}$ [fm], $a_0 = {a0}$ [fm]", fontsize=8)

plt.show()



# Database

In [ ]:
from luescher_nd.database.utilities import read_table
import pandas as pd
import os

In [ ]:
ROOT = os.path.abspath("../data/two-d")
db = os.path.join(ROOT, "db-contact-fv-c-parity.sqlite")

In [ ]:
df = read_table(database=db, drop_comment=False).drop(columns="y")

df = (
    pd.merge(
        df,
        df.comment.str.extract(
            "spherical=(?P<spherical>[01]{1})&a0=(?P<a0>[0-9\.]+)&n=(?P<n>[0-9]+)"
        ),
        how="inner",
        left_index=True,
        right_index=True,
    )
    .astype(dtype={"spherical": int, "a0": float, "n": int})
    .astype({"spherical": bool})
)


zeta = {True: z_spherical, False: z_cartesian}


df["y"] = df.apply(
    lambda row: ere(zeta[row["spherical"]](row["x"]), row["x"])[0], axis=1
)
df = df.dropna()

df = df.drop(columns=["comment", "date"])
df["epsilon"] = df.epsilon.round(3)

tf = df.query("y < 5 and y > -5 and epsilon < 0.1 and L == 1").sort_values(
    ["spherical", "n1d", "nstep", "nlevel"]
)

tf.head()


In [ ]:
grid = sns.FacetGrid(
    data=tf,
    row="spherical",
    col="nstep",
    hue="epsilon",
    legend_out=True,
    margin_titles=True,
    col_order=["1", "4", "$\\infty$"],
    hue_kws=dict(marker=["o", "s", "v", "^", "d"])
)

grid.map(plt.plot, "x", "y", ms=3, ls=":", marker="s", lw=0.5).add_legend(title=r"$\epsilon$ [fm]")

x = np.linspace(-0.1, 20, 200)

for ax in grid.axes.flatten():
    ax.set_ylim(-1, 3)

    ax.axhline(0, color="black", lw=0.5, ls="-", zorder=-1)
    ax.axvline(0, color="black", lw=0.5, ls="-", zorder=-1)
    ax.plot(
        x,
        2 / np.pi * np.log(a0 * 2 * np.pi * np.sqrt(x) / L),
        color="black",
        ls="--",
        lw="1",
        label="Analytic",
        zorder=-1,
    )

grid.set_ylabels("$\cot(\delta_0(x))$")
grid.set_xlabels("$x$")

grid.fig.suptitle(f"$L = {tf.L.unique()[0]}$ [fm], $a_0 = {tf.a0.unique()[0]}$ [fm]", y=1.06)

grid.fig.set_dpi(250)
grid.fig.set_figheight(4)

In [ ]:
cf = df.groupby(["epsilon", "L", "nstep", "spherical"])[["contact_strength"]].mean().reset_index()#.query("L == 1")

cf["y"] = -1./cf["contact_strength"]

cf.head()

In [ ]:
grid = sns.FacetGrid(
    data=cf,
    row="spherical",
    col="nstep",
    hue="L",
    legend_out=True,
    margin_titles=True,
    col_order=["1", "4", "$\\infty$"],
    hue_kws=dict(marker=["o", "s", "v", "^", "d"]),
)

grid.map(plt.plot, "epsilon", "y", ms=3, ls=":", marker="s", lw=0.5)

x = np.linspace(-0.1, 20, 200)

grid.set_ylabels(r"$-1/c_0(\epsilon)$ [fm$^{-1}$]")
grid.set_xlabels("$\epsilon$ [fm]")

grid.fig.suptitle(f"$a_0 = {df.a0.unique()[0]}$ [fm]", y=1.06)

eps = np.linspace(1.0e-2, 9.0e-1, 100)
Lambda = 2* np.pi / eps
y1 = 2 * mu / np.pi / 2 * np.log(a0 * Lambda)
G = 0.52326651247849486024677309202244625024950319935978
y2 = y1 - 2 * mu / np.pi**2 * (G - np.pi/8)

for ax in grid.axes.flatten():
    ax.set_xscale("log")
    ax.set_xlim(1.0e-2, 9.0e-1)
    ax.plot(
        eps,
        y1,
        label=r"$\frac{\mu}{\pi} \log\left(\frac{2\pi}{\epsilon}a_0\right)$",
        ls="-",
        color="blue",
        lw=.5,
        zorder=-1
    )
    ax.plot(
        eps,
        y2,
        label=r"$\frac{\mu}{\pi} \log\left(\frac{2\pi}{\epsilon}a_0\right) "
        r"- \frac{2\mu}{\pi^2}\left(G - \frac{\pi}{8}\right)$",
        ls="-",
        color="green",
        lw=.5,
        zorder=-1
    )

grid.fig.set_dpi(250)
grid.fig.set_figheight(4)

plt.subplots_adjust()

leg = ax.legend(fontsize=10, frameon=False, loc="upper left", bbox_to_anchor=(1.1, 1.5), title="$L$ [fm]")


In [ ]:
ttf = cf.query("nstep == '$\\infty$' and spherical == True")
Lambda = 2* np.pi / ttf.epsilon
y1 = 2 * mu / np.pi / 2 * np.log(a0 * Lambda)

fig, ax = plt.subplots(figsize=(3,2), dpi=200)

plt.plot(ttf.epsilon, np.abs(ttf.y - y1)/y1 * 100, ls="--", lw=.5, marker="o", ms=1)
sns.despine()

ax.set_xlabel("$\epsilon$ [fm]")
ax.set_ylabel(r"$ \left|\Delta\frac{-1}{c_0(\epsilon)}\right|$ [%]")
ax.set_title("Spherical data, spherical ERE," f" nstep = {ttf.nstep.unique()[0]}", fontsize=8)

ax.set_yscale("log")

plt.show()